# Merged results filtering - part 1.


Since crisprtools cannot offer the assembly info, I decided to make a custom Python parser.

In [1]:
location = "/home/sergiu/data/data/work/andersson/data/merged/crass.crispr"
import xmltodict
with open(location,'r') as fd:
    doc = xmltodict.parse(fd.read())
print "Done!"

Done!


In [8]:
doc[u'crispr'][u'group'][0]

OrderedDict([(u'@drseq', u'GGTTCATCCCCGCTTGTGCGGGGGAGAC'),
             (u'@gid', u'G14'),
             (u'data',
              OrderedDict([(u'sources',
                            OrderedDict([(u'source',
                                          [OrderedDict([(u'@accession',
                                                         u'120314_R1:HISEQ:99:D2GAAACXX:3:2313:10117:58716'),
                                                        (u'@soid', u'SO2')]),
                                           OrderedDict([(u'@accession',
                                                         u'120314_R1:HISEQ:99:D2GAAACXX:3:2104:5332:22621'),
                                                        (u'@soid', u'SO7')]),
                                           OrderedDict([(u'@accession',
                                                         u'121220_R1:HISEQ:99:D2GAAACXX:2:2301:14584:7872'),
                                                        (u'@soid', u'SO12')]),
              

In [9]:
len(doc[u'crispr'][u'group'])

2971

In [10]:
doc[u'crispr'][u'group'][0][u'@gid']

u'G14'

Here I am outing the internal accession id and the external sample id.

In [14]:
doc[u'crispr'][u'group'][0][u'data'][u'sources'][u'source'][0]

OrderedDict([(u'@accession',
              u'120314_R1:HISEQ:99:D2GAAACXX:3:2313:10117:58716'),
             (u'@soid', u'SO2')])

In [16]:
doc[u'crispr'][u'group'][0].keys()

[u'@drseq', u'@gid', u'data', u'metadata', u'assembly']

In [18]:
doc[u'crispr'][u'group'][0][u'assembly'][u'contig'][0]

OrderedDict([(u'@cid', u'C1'),
             (u'cspacer',
              [OrderedDict([(u'@spid', u'SP10'),
                            (u'fspacers',
                             OrderedDict([(u'fs',
                                           OrderedDict([(u'@drconf', u'0'),
                                                        (u'@drid', u'DR1'),
                                                        (u'@spid',
                                                         u'SP235')]))]))]),
               OrderedDict([(u'@spid', u'SP235'),
                            (u'bspacers',
                             OrderedDict([(u'bs',
                                           OrderedDict([(u'@drconf', u'0'),
                                                        (u'@drid', u'DR1'),
                                                        (u'@spid',
                                                         u'SP10')]))])),
                            (u'fspacers',
                             Or

Let's now get a table such as this:

```
                             Sample1   Sample2 ...
CRISPR1 Spacer1              11               0         ...
CRISPR1 Spacer2               0                3         ...
CRISPR2 Spacer3               1                7         ...
```

In [25]:
sample_names = ['120314_R1', '120322_R1', '120328_R1', '120403_R1', '120416_R1', '120419_R1', '120423_R1', '120507_R1', '120516_R1', '120521_R1', '120531_R1', '120604_R1', '120613_R1', '120619_R1', '120628_R1', '120705_R1', '120709_R1', '120717_R1', '120802_R1', '120806_R1', '120813_R1', '120820_R1', '120823_R1', '120828_R1', '120903_R1', '120910_R1', '120920_R1', '120924_R1', '121001_R1', '121004_R1', '121015_R1', '121022_R1', '121028_R1', '121105_R1', '121123_R1', '121128_R1', '121220_R1', '120314_R2', '120322_R2', '120328_R2', '120403_R2', '120416_R2', '120419_R2', '120423_R2', '120507_R2', '120516_R2', '120521_R2', '120531_R2', '120604_R2', '120613_R2', '120619_R2', '120628_R2', '120705_R2', '120709_R2', '120717_R2', '120802_R2', '120806_R2', '120813_R2', '120820_R2', '120823_R2', '120828_R2', '120903_R2', '120910_R2', '120920_R2', '120924_R2', '121001_R2', '121004_R2', '121015_R2', '121022_R2', '121028_R2', '121105_R2', '121123_R2', '121128_R2', '121220_R2']
samples = {s:i for i,s in enumerate(sample_names)}
f = open("/home/sergiu/data/data/work/andersson/data/spacer_table.txt","w")
data = ["CRISPRid","DRseq","SPid","SPseq"] + sample_names
f.write("\t".join(data)+'\n')
for group in doc[u'crispr'][u'group'][:]:
    gid = group[u'@gid']
    drseq = group[u'@drseq']
    sources = {}
    for source in group[u'data'][u'sources'][u'source']:
        sources[source[u'@soid']] = source[u'@accession']
    spacers = {}
    for spacer in group[u'data'][u'spacers'][u'spacer']:
        spid = spacer[u'@spid']
        spseq = spacer[u'@seq']
        if len(spacer[u'source']) > 1:
            soids = set([s[u'@soid'] for s in spacer[u'source']])
        else:
            soids = set([spacer[u'source'][u'@soid']])
        spcounts = [0 for i in sample_names]
        for soid in soids:
            #print soid
            #print sources[soid]
            sampleid = sources[soid].split(':')[0]
            spcounts[samples[sampleid]] += 1
        #print gid, drseq, spid, spseq, spcounts
        data = [gid, drseq, spid, spseq] + [str(c) for c in spcounts]
        f.write("\t".join(data)+'\n')
f.close()
print "Done!"

Done!


In [5]:
doc[u'crispr'][u'group'][0][u'data'][u'sources'][u'source'][0][u'@accession']

u'120314_R1:HISEQ:99:D2GAAACXX:3:2313:10117:58716'

In [10]:
doc[u'crispr'][u'group'][0][u'@gid']

u'G14'

In [13]:
doc[u'crispr'][u'group'][0][u'data'][u'spacers'][u'spacer'][0]

OrderedDict([(u'@cov', u'1'),
             (u'@seq', u'CGCGGAATGATCTTATCACCTGGGCCATCGTGG'),
             (u'@spid', u'SP5'),
             (u'source',
              [OrderedDict([(u'@soid', u'SO2')]),
               OrderedDict([(u'@soid', u'SO273')])])])

Some of the spacers have unusually high counts and also very similar profiles across every sample. The problem was formulated by Anders:

Some regions of the table look strange. For instance rows 215 - 232 (start of cluster G1907). All of these 18 spacers have nearly identical distribution patterns over the samples (147 or 148 copies in the first sample, 2 copies in the second sample, 0 in the third, etc). This can't happen by chance, and all of them can't be captured on the same reads. All of these spacers share their first 9 bases, and these are actually part of the repeat. 

```
ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC
---------------AGCGCTCGG---------------------
```

Not sure what CRASS has done here but there seems to be quite a few of those cases. Probably worth investigating this before making graphs.


In [7]:
psfile = "/home/sergiu/data/data/work/andersson/data/verify/problem_spacers.txt"
import pandas as pd
#with open(psfile,'r') as f:
dt = pd.read_table(psfile)

In [8]:
dt['SPid'].values

array(['SP1217', 'SP149', 'SP1127', 'SP582', 'SP5449', 'SP5757', 'SP590',
       'SP1212', 'SP3350', 'SP2273', 'SP1225', 'SP1826', 'SP2004',
       'SP2734', 'SP5361', 'SP4115', 'SP7304', 'SP7676'], dtype=object)

In [4]:
dt

,CRISPRid,DRseq,SPid,SPseq,120314_R1,120322_R1,120328_R1,120403_R1,120416_R1,120419_R1,...,120924_R2,121001_R2,121004_R2,121015_R2,121022_R2,121028_R2,121105_R2,121123_R2,121128_R2,121220_R2
0,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP1217,AGCGCTCGGCCCCCTAGCCTGCCTAGCCGCAGCGCTCGGCCCCCTA...,148,2,0,1,2,5,...,3,0,0,1,1,52,12,2,4,11
1,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP149,AGCGCTCGGCCCCAGCCCCCAATATAAACTCACCTAACCCAATATA...,148,2,0,1,2,5,...,5,2,3,1,1,61,12,2,4,11
2,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP1127,AGCGCTCGGCCCCCTAGCCTGCCTAGCCGCGGGCGCTCGGCCCCAG...,148,2,0,1,2,5,...,3,0,0,1,1,52,12,2,4,11
3,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP582,AGCGCTCGGCCCCAGCCACCCAAAATAAACTCACCTAACCCGATAT...,147,2,0,1,2,5,...,3,0,0,1,1,52,12,2,4,11
4,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP5449,AGCGCTCGGCCCCAGCCCCCAATATAAACTCACCTAACCCAATATA...,148,2,0,1,2,5,...,3,0,0,1,1,53,12,2,4,11
5,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP5757,AGCGCTCGGCCCCAGCCCCCAATATAAACTCACCTAACCCAATATA...,147,2,0,1,2,5,...,6,0,0,2,1,54,17,2,4,11
6,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP590,AGCGCTCGGCCCCAGCCCCCAATATAACTTAACCAAACCCATTTTT...,147,2,0,1,2,5,...,3,0,0,1,1,53,12,2,4,11
7,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP1212,AGCGCTCGGCCCCAGCCCCCAATATAAACTCACCTAACCCAATATA...,147,2,0,1,2,5,...,3,0,0,1,1,52,12,2,4,11
8,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP3350,AGCGCTCGGCCCCAGCCCCCAATATAAACTCACCTAACCCAATATA...,147,2,0,1,2,5,...,3,0,0,1,1,53,12,2,4,11
9,G1907,ACCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCCGC,SP2273,AGCGCTCGGCCCCAGCCCCCAATATAACCTAACCTAACC,147,2,0,1,2,5,...,3,0,0,1,1,53,12,2,4,11


In [5]:
"""
- if some of the spacers in group G1907 are inside an assembled contig
- print the contig sequence of spacers
- print the reads specific to a contig
"""
for igroup, group in enumerate(doc[u'crispr'][u'group'][:]):
    gid = group[u'@gid']
    if gid == "G1907":
        print igroup
        drseq = group[u'@drseq']
        sources = {}
        for source in group[u'data'][u'sources'][u'source']:
            sources[source[u'@soid']] = source[u'@accession']
        spacers = {}
        for spacer in group[u'data'][u'spacers'][u'spacer']:
            spid = spacer[u'@spid']
            spseq = spacer[u'@seq']
            if len(spacer[u'source']) > 1:
                soids = set([s[u'@soid'] for s in spacer[u'source']])
            else:
                soids = set([spacer[u'source'][u'@soid']])
            spacers[spid] = [sources[soid] for soid in soids]
print "Done!"

12
Done!


In [38]:
#doc[u'crispr'][u'group'][12][u'assembly'][u'contig'][0][u'cspacer'][0][u'@spid']
doc[u'crispr'][u'group'][12][u'assembly'][u'contig'][0][u'cspacer']
#doc[u'crispr'][u'group'][0][u'assembly'][u'cspacer'][0][u'@spid']

[OrderedDict([(u'@spid', u'FL6'),
              (u'fspacers',
               OrderedDict([(u'fs',
                             OrderedDict([(u'@drconf', u'0'),
                                          (u'@drid', u'DR1'),
                                          (u'@spid', u'FL5119')]))]))]),
 OrderedDict([(u'@spid', u'SP5119'),
              (u'bflankers',
               OrderedDict([(u'bf',
                             OrderedDict([(u'@directjoin', u'0'),
                                          (u'@drconf', u'0'),
                                          (u'@flid', u'SP6')]))]))])]

In [20]:
doc[u'crispr'][u'group'][12][u'data'][u'spacers'][u'spacer']

[OrderedDict([(u'@cov', u'8'),
              (u'@seq',
               u'AGCGCTCGGCCCCCTAGCCTGCCTAGCCGCAGCGCTCGGCCCCCTAGCCTGCCTAGCC'),
              (u'@spid', u'SP1217'),
              (u'source',
               [OrderedDict([(u'@soid', u'SO2')]),
                OrderedDict([(u'@soid', u'SO7')]),
                OrderedDict([(u'@soid', u'SO9')]),
                OrderedDict([(u'@soid', u'SO10')]),
                OrderedDict([(u'@soid', u'SO16')]),
                OrderedDict([(u'@soid', u'SO19')]),
                OrderedDict([(u'@soid', u'SO20')]),
                OrderedDict([(u'@soid', u'SO21')]),
                OrderedDict([(u'@soid', u'SO22')]),
                OrderedDict([(u'@soid', u'SO24')]),
                OrderedDict([(u'@soid', u'SO27')]),
                OrderedDict([(u'@soid', u'SO29')]),
                OrderedDict([(u'@soid', u'SO41')]),
                OrderedDict([(u'@soid', u'SO42')]),
                OrderedDict([(u'@soid', u'SO45')]),
                OrderedDic

In [23]:
contigs

[[u'FL6', u'SP5119'], [], []]

In [24]:
spacers['SP5119']

[u'120531_R2:HISEQ:99:D2GAAACXX:4:1209:20889:10087',
 u'120619_R2:HISEQ:99:D2GAAACXX:1:1115:16986:33072',
 u'120619_R2:HISEQ:159:H72EAADXX:1:1104:21066:30214',
 u'120619_R1:HISEQ:99:D2GAAACXX:1:1210:17070:2293',
 u'120531_R1:HISEQ:99:D2GAAACXX:4:2206:3978:2921',
 u'120619_R2:HISEQ:159:H72EAADXX:2:1215:15842:17350',
 u'120314_R1:HISEQ:99:D2GAAACXX:3:1315:17111:63252',
 u'120619_R2:HISEQ:99:D2GAAACXX:1:2207:15262:72057',
 u'120619_R2:HISEQ:99:D2GAAACXX:1:1306:3830:14733',
 u'120619_R1:HISEQ:99:D2GAAACXX:1:1113:11033:85713',
 u'120314_R2:HISEQ:99:D2GAAACXX:3:2109:14790:41621',
 u'120531_R2:HISEQ:99:D2GAAACXX:4:2210:8128:55666']

In [9]:
with open('/home/sergiu/data/data/work/andersson/data/verify/problem_spacers_accessions.txt','w') as f:
    for sp in dt['SPid'].values:
        f.write('\t'.join([sp]+spacers[sp])+'\n')
    

The mail I sent was:

Trying to prepare the data for you I managed to also solve the problem, it seems that I was in fact looking at the wrong assembly, the only sequence that could be assembled in this group is: FL1 + DR + SP5119, so one flanker, the direct read and one spacer.

The 'bad' spacers with multiple counts could in fact not be assembled. The only good guy is mapped to (fasta file attached):
```
SP5119:
[u'120531_R2:HISEQ:99:D2GAAACXX:4:1209:20889:10087',
 u'120619_R2:HISEQ:99:D2GAAACXX:1:1115:16986:33072',
 u'120619_R2:HISEQ:159:H72EAADXX:1:1104:21066:30214',
 u'120619_R1:HISEQ:99:D2GAAACXX:1:1210:17070:2293',
 u'120531_R1:HISEQ:99:D2GAAACXX:4:2206:3978:2921',
 u'120619_R2:HISEQ:159:H72EAADXX:2:1215:15842:17350',
 u'120314_R1:HISEQ:99:D2GAAACXX:3:1315:17111:63252',
 u'120619_R2:HISEQ:99:D2GAAACXX:1:2207:15262:72057',
 u'120619_R2:HISEQ:99:D2GAAACXX:1:1306:3830:14733',
 u'120619_R1:HISEQ:99:D2GAAACXX:1:1113:11033:85713',
 u'120314_R2:HISEQ:99:D2GAAACXX:3:2109:14790:41621',
 u'120531_R2:HISEQ:99:D2GAAACXX:4:2210:8128:55666']
```
The bad spacers map to many many many reads, their full mappings are also attached (tab separated).

This brings back the filtering questions we were talking about at the last meeting, based on correct assemblies. I am going to prepare a filtered table today, where I only take into account spacers for whom there is a specified assembly order (no assembly is in fact carried as you may recall, but an order is specified that derives from the p-graphs). It will still not contain any info on the actual spacer position, but at least disregards all the floating spacers out there.

However, a floating spacer is still something that stands between two DRs, how wise is it to remove them all? 

In [39]:
"""This outputs the counts table only for the spacers that have a specified assembly order"""
sample_names = ['120314_R1', '120322_R1', '120328_R1', '120403_R1', '120416_R1', '120419_R1', '120423_R1', '120507_R1', '120516_R1', '120521_R1', '120531_R1', '120604_R1', '120613_R1', '120619_R1', '120628_R1', '120705_R1', '120709_R1', '120717_R1', '120802_R1', '120806_R1', '120813_R1', '120820_R1', '120823_R1', '120828_R1', '120903_R1', '120910_R1', '120920_R1', '120924_R1', '121001_R1', '121004_R1', '121015_R1', '121022_R1', '121028_R1', '121105_R1', '121123_R1', '121128_R1', '121220_R1', '120314_R2', '120322_R2', '120328_R2', '120403_R2', '120416_R2', '120419_R2', '120423_R2', '120507_R2', '120516_R2', '120521_R2', '120531_R2', '120604_R2', '120613_R2', '120619_R2', '120628_R2', '120705_R2', '120709_R2', '120717_R2', '120802_R2', '120806_R2', '120813_R2', '120820_R2', '120823_R2', '120828_R2', '120903_R2', '120910_R2', '120920_R2', '120924_R2', '121001_R2', '121004_R2', '121015_R2', '121022_R2', '121028_R2', '121105_R2', '121123_R2', '121128_R2', '121220_R2']
samples = {s:i for i,s in enumerate(sample_names)}
f = open("/home/sergiu/data/data/work/andersson/data/spacer_table_filtered.txt","w")
data = ["CRISPRid","DRseq","SPid","SPseq"] + sample_names
f.write("\t".join(data)+'\n')
for igroup, group in enumerate(doc[u'crispr'][u'group'][:]):
    gid = group[u'@gid']
    drseq = group[u'@drseq']
    sources = {}
    for source in group[u'data'][u'sources'][u'source']:
        sources[source[u'@soid']] = source[u'@accession']
    spacers = {}
    for spacer in group[u'data'][u'spacers'][u'spacer']:
        spid = spacer[u'@spid']
        spseq = spacer[u'@seq']
        if len(spacer[u'source']) > 1:
            soids = set([s[u'@soid'] for s in spacer[u'source']])
        else:
            soids = set([spacer[u'source'][u'@soid']])
        spacers[spid] = [sources[soid] for soid in soids]
    contigs = []
    spcont = set() # the set of spacers lying in contigs
    for contig in group[u'assembly'][u'contig']:
        slist = []
        if u'cspacer' in contig:
            if u'@spid' in contig[u'cspacer']:
                slist.append(contig[u'cspacer'][u'@spid'])
                spcont.add(contig[u'cspacer'][u'@spid'])
            else:
                for spacer in contig[u'cspacer']:
                    slist.append(spacer[u'@spid'])
                    spcont.add(spacer[u'@spid'])
        contigs.append(slist)
    # Now write the table    
    for spacer in group[u'data'][u'spacers'][u'spacer']:
        spid = spacer[u'@spid']
        if spid in spcont:
            spseq = spacer[u'@seq']
            if len(spacer[u'source']) > 1:
                soids = set([s[u'@soid'] for s in spacer[u'source']])
            else:
                soids = set([spacer[u'source'][u'@soid']])
            spcounts = [0 for i in sample_names]
            for soid in soids:
                #print soid
                #print sources[soid]
                sampleid = sources[soid].split(':')[0]
                spcounts[samples[sampleid]] += 1
            #print gid, drseq, spid, spseq, spcounts
            data = [gid, drseq, spid, spseq] + [str(c) for c in spcounts]
            f.write("\t".join(data)+'\n')

f.close()
print "Done!"

Done!


In [32]:
spacer

u'@spid'

In [36]:
contig[u'cspacer']

OrderedDict([(u'@spid', u'SP120'),
             (u'bspacers',
              OrderedDict([(u'bs',
                            OrderedDict([(u'@drconf', u'0'),
                                         (u'@drid', u'DR1'),
                                         (u'@spid', u'SP5')]))])),
             (u'fspacers',
              OrderedDict([(u'fs',
                            [OrderedDict([(u'@drconf', u'0'),
                                          (u'@drid', u'DR1'),
                                          (u'@spid', u'SP319')]),
                             OrderedDict([(u'@drconf', u'0'),
                                          (u'@drid', u'DR1'),
                                          (u'@spid', u'SP84')])])]))])

Anders replied that the problem spacers in G1907 seem to share a lot of reads. Fast check:

In [11]:
spd = {}
with open('/home/sergiu/data/data/work/andersson/data/verify/problem_spacers_accessions.txt','r') as f:
    for l in f:
        r = l.strip('\n').split('\t')
        sp = r[0]
        reads = set(r[1:])
        spd[sp] = reads
for sp1 in spd:
    for sp2 in spd:
        if not sp1 == sp2:
            print sp1, sp2, len(spd[sp1]&spd[sp2]), len(spd[sp1]|spd[sp2]), len(spd[sp1]&spd[sp2])/float(len(spd[sp1]|spd[sp2]))
            

SP5361 SP582 1576 1600 0.985
SP5361 SP149 1576 1668 0.9448441247
SP5361 SP1225 1576 1590 0.991194968553
SP5361 SP590 1576 1586 0.99369482976
SP5361 SP3350 1576 1586 0.99369482976
SP5361 SP1212 1576 1585 0.994321766562
SP5361 SP1127 1576 1591 0.990571967316
SP5361 SP7304 1576 1586 0.99369482976
SP5361 SP1217 1576 1596 0.987468671679
SP5361 SP1826 1576 1592 0.989949748744
SP5361 SP2273 1576 1618 0.974042027194
SP5361 SP7676 1576 1584 0.994949494949
SP5361 SP5449 1576 1608 0.980099502488
SP5361 SP2734 1576 1584 0.994949494949
SP5361 SP5757 1576 1648 0.956310679612
SP5361 SP2004 1576 1597 0.986850344396
SP5361 SP4115 1576 1584 0.994949494949
SP582 SP5361 1576 1600 0.985
SP582 SP149 1576 1678 0.939213349225
SP582 SP1225 1576 1600 0.985
SP582 SP590 1576 1596 0.987468671679
SP582 SP3350 1576 1596 0.987468671679
SP582 SP1212 1576 1595 0.988087774295
SP582 SP1127 1576 1601 0.984384759525
SP582 SP7304 1576 1596 0.987468671679
SP582 SP1217 1576 1606 0.981320049813
SP582 SP1826 1576 1602 0.9837702

The overlap is 99%. Also each spacer is mapping to thousands of reads which is not what normally happens.

TODO:

The spacers in CRISPR clusters are mostly sparse, other observed cases include:
- some spacers share many reads and we were expecting none to be shared.
- some spacers are mapped to many thousands of reads while others are barely mapped.
- some spacers are distributed in a very similar manner.
- some spacers have no chain assembly, being described as "floating". Do they have a DR in both ends?

Unanswered question include:
- are these the same spacers that share all these features?

Anders observed something more:

> And looking at the filtered table that you sent there still seem to be problematic spacers. For example the spacers with highest counts in the table, in cluster G25151, have suspiciously correlated abundance profiles. Four of them share a sequence in the start and 2 of them instead share a sequence in the end. 

In [2]:
for igroup, group in enumerate(doc[u'crispr'][u'group'][:]):
    gid = group[u'@gid']
    if gid == "G25151":
        print igroup
        drseq = group[u'@drseq']
        sources = {}
        for source in group[u'data'][u'sources'][u'source']:
            sources[source[u'@soid']] = source[u'@accession']
        spacers = {}
        for spacer in group[u'data'][u'spacers'][u'spacer']:
            spid = spacer[u'@spid']
            spseq = spacer[u'@seq']
            if len(spacer[u'source']) > 1:
                soids = set([s[u'@soid'] for s in spacer[u'source']])
            else:
                soids = set([spacer[u'source'][u'@soid']])
            spacers[spid] = [sources[soid] for soid in soids]
print "Done!"

190
Done!


In [8]:
for sp1 in spacers.keys():
    for sp2 in spacers.keys():
        if not sp1 == sp2:
            print sp1, sp2, len(set(spacers[sp1])&set(spacers[sp2])), len(set(spacers[sp1])|set(spacers[sp2])), \
            len(set(spacers[sp1])&set(spacers[sp2]))/float(len(set(spacers[sp1])|set(spacers[sp2])))

SP783 SP1390 88 537 0.163873370577
SP783 SP957 1 388 0.00257731958763
SP783 SP23 372 536 0.694029850746
SP783 SP1368 86 541 0.158964879852
SP783 SP615 2 376 0.00531914893617
SP783 SP373 372 398 0.934673366834
SP783 SP45 372 418 0.88995215311
SP1390 SP783 88 537 0.163873370577
SP1390 SP957 0 266 0.0
SP1390 SP23 248 537 0.461824953445
SP1390 SP1368 248 256 0.96875
SP1390 SP615 0 255 0.0
SP1390 SP373 86 561 0.153297682709
SP1390 SP45 88 579 0.151986183074
SP957 SP783 1 388 0.00257731958763
SP957 SP1390 0 266 0.0
SP957 SP23 1 548 0.00182481751825
SP957 SP1368 0 268 0.0
SP957 SP615 0 19 0.0
SP957 SP373 1 410 0.00243902439024
SP957 SP45 4 427 0.00936768149883
SP23 SP783 372 536 0.694029850746
SP23 SP1390 248 537 0.461824953445
SP23 SP957 1 548 0.00182481751825
SP23 SP1368 248 539 0.460111317254
SP23 SP615 2 536 0.00373134328358
SP23 SP373 372 558 0.666666666667
SP23 SP45 374 576 0.649305555556
SP1368 SP783 86 541 0.158964879852
SP1368 SP1390 248 256 0.96875
SP1368 SP957 0 268 0.0
SP1368 SP23

Some spacer pairs that have over 0.5 reads in common:
SP23 SP45
SP783 SP45
SP783 SP23

SP1368 SP1390



In [65]:
"""Just getting the spacer sequence *rolls eyes"""
def get_gind(gid, doc):
    ig = -1
    g = None
    for ig, g in enumerate(doc[u'crispr'][u'group']):
        if g[u'@gid'] == "G25151":
           break
    return ig

def get_spind(spid, group):
    isp = -1
    sp = None
    for isp, sp in enumerate(group[u'data'][u'spacers'][u'spacer']):
        if sp[u'@spid'] == spid:
            spseq = sp[u'@seq']
            break
    return isp

ig = get_gind("G25151", doc)
print "Group:"
print doc[u'crispr'][u'group'][ig][u'@gid']
print doc[u'crispr'][u'group'][ig][u'@drseq']
drseq = doc[u'crispr'][u'group'][ig][u'@drseq']

splist = "SP1368 SP1390".split()
spinds = [get_spind(sp, doc[u'crispr'][u'group'][ig]) for sp in splist]
spseqs = []
print "Spacer sequences:"
for spind in spinds:
    print doc[u'crispr'][u'group'][ig][u'data'][u'spacers'][u'spacer'][spind][u'@spid']
    print doc[u'crispr'][u'group'][ig][u'data'][u'spacers'][u'spacer'][spind][u'@seq']
    spseqs.append(doc[u'crispr'][u'group'][ig][u'data'][u'spacers'][u'spacer'][spind][u'@seq'])

doc[u'crispr'][u'group'][ig][u'assembly'][u'contig']

Group:
G25151
ACAGAGACCGCGCGCTTGTGCTTTGGTGGTGCACCCTGGATGC
Spacer sequences:
SP1368
TCGCACTTTGACCGTGTGCCGCCTCCATGCTTGCA
SP1390
CCACATACAGTCCGTGTGCCGCCTCCATGCTTGCA


[OrderedDict([(u'@cid', u'C1'),
              (u'cspacer',
               [OrderedDict([(u'@spid', u'SP45'),
                             (u'fspacers',
                              OrderedDict([(u'fs',
                                            OrderedDict([(u'@drconf', u'0'),
                                                         (u'@drid', u'DR1'),
                                                         (u'@spid',
                                                          u'SP1368')]))]))]),
                OrderedDict([(u'@spid', u'SP1368'),
                             (u'bspacers',
                              OrderedDict([(u'bs',
                                            OrderedDict([(u'@drconf', u'0'),
                                                         (u'@drid', u'DR1'),
                                                         (u'@spid',
                                                          u'SP45')]))]))])])]),
 OrderedDict([(u'@cid', u'C2'),
              (u'

In [15]:
gid = "G25151"
splist = "SP1368 SP1390".split()
location = "/home/sergiu/data/data/work/andersson/data/merged/Group_" + gid[1:] + "_" + drseq + ".fa"
readlist = []
reads = set(spacers[splist[0]])
for sp in splist[1:]:
    reads &= set(spacers[sp])
with open(location, 'r') as f:
    rid = ""
    r = ""
    for l in f:
        if l[0] == ">":
            rid = l[1:].split()[0]
            if rid in reads and not len(r)==0:
                print rid
                print r
                readlist.append((rid,r))
            r = ""
        else:
            r += l.strip("\n")
            

120322_R1:HISEQ:99:D2GAAACXX:3:2307:17200:80038
CCACGCGCTTGTGCTTTGGTGGTGCACCCTGCATGTCCGCACACAGACCGTGCGTCGCCTCCATGCTTGACACAGAGACCATGCGCTTGTGCGTTGGTGGT
120322_R1:HISEQ:99:D2GAAACXX:3:1210:20166:21249
CCGCGCGCTTGTGCTTTGGTGGTGCACCCTGCAAATCCGCACACAGACCGTGTATCACCCCCATGCTTGCAACAGAGACCGCGTGTTTGTGCTTTGGTGGT
120322_R1:HISEQ:99:D2GAAACXX:3:1112:11459:44297
CGTCACCTCCATGCTTGCGACAGAGACCGTGCGCTTGTGCTTTGGTGGTGCACCCTTGATGCCCGCACACAGTCCGTGTACCGCCTCCATGCTTGCAACAG
120322_R1:HISEQ:99:D2GAAACXX:3:2309:4096:67264
GGGGCACCCTGGAAACACGCATGCCCCTTTGCCCCCATGCTTGCAACAGAGACCGCGCGCTTGTGCTTTGGTGGTGCACCCTGCAAATCCGCACACAGACC
120322_R1:HISEQ:99:D2GAAACXX:3:1313:6056:76759
GGGGCACCCTGGAAACACGCATGCCCCTTTGCCCCCATGCTTGCAACAGAGACCGCGCGCTTGTGCTTTGGTGGTGCACCCTGCAAATCCGCACACAGACC
121220_R1:HISEQ:99:D2GAAACXX:2:2216:20699:96038
GTAATATGTATGCGGATGCCCATGCTTAACACAGAGACCGCGCGCTTGTGCTTTGGTGGTGCACCCTGCATGTCCGCACACCCCCTTCGTACTGCCTCCAT
121220_R1:HISEQ:164:H72E2ADXX:1:1207:2786:41726
TGCGCTGGCGGTGCACCCTGCATGTCCGCACACCCCCTTCGTACTGCCTCCATG

There are just so many of these reads! How would they align the DR sequence?

In [14]:
# Testing ..
from Bio import pairwise2
alignments = pairwise2.align.globalxx("ACCGT", "ACG")
from Bio.pairwise2 import format_alignment
print(format_alignment(*alignments[0]))

ACCGT
|||||
A-CG-
  Score=3



In [62]:
%%capture cap --no-stderr
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

for rid, r in readlist:
    print "\n"
    print "Read:"
    print ">" + rid
    print r
    print ""
    for sp, seq in zip(splist, spseqs):
        print "Spacer alignment for:", sp, seq
        alignments = pairwise2.align.globalxx(r, seq)
        print(format_alignment(*alignments[0]))
    print "DR alignment:", drseq
    alignments = pairwise2.align.globalxx(r, drseq)
    print(format_alignment(*alignments[0]))
    

In [53]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
spseq = "CCGCCCACAGACCGTGTGTCGCCTCCATGCTTGTA"
drseq = "ACAGAGACCGCGCGCTTGTGCTTTGGTGGTGCACCCTGGATGC"
for rid, r in readlist:
    print ">" + rid
    print r
    print "DR alignments:"
    alignments = pairwise2.align.globalxx(r, spseq)
    print(format_alignment(*alignments[0]))

In [63]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)